In [24]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def selecionar_melhor_k(ks, X_treino, X_val, y_treino, y_val):
    acuracias_val = []
    for k in ks:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_treino, y_treino)
        pred = knn.predict(X_val)
        acuracias_val.append(accuracy_score(y_val, pred))

    melhor_val = max(acuracias_val)
    melhor_k = ks[np.argmax(acuracias_val)]

    knn = KNeighborsClassifier(n_neighbors=melhor_k)
    knn.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])
    
    return knn, melhor_k, melhor_val

In [3]:
ds = load_iris(as_frame=True)
df = ds.frame
X = ds.data.values
y = ds.target.values

In [26]:
folds = 10
k2=5
skf = StratifiedKFold(n_splits=folds)

acuracias = []

for idx_treino, idx_teste in skf.split(X,y):
    X_treino = X[idx_treino]
    y_treino = y[idx_treino]

    X_teste = X[idx_teste]
    y_teste = y[idx_teste]
    
    ss=StandardScaler()
    ss.fit(X_treino)
    X_treino = ss.transform(X_treino)
    X_val = ss.transform(X_val)
    X_teste = ss.transform(X_teste)
    
    params = {"n_neighbors" : range(1,30,2)}
    knn = KNeighborsClassifier()
    knn = GridSearchCV(knn,params,cv=StratifiedKFold(n_splits=k2))
    knn.fit(X_treino,y_treino)
    
    
    acuracias.append(accuracy_score(y_teste,knn.predict(X_teste)))


In [27]:
"%.2f +- %.2f (min: %.2f, max: %.2f)" % (np.mean(acuracias), np.std(acuracias), min(acuracias), max(acuracias))

'0.95 +- 0.05 (min: 0.87, max: 1.00)'

In [28]:
acuracias

[1.0,
 0.9333333333333333,
 1.0,
 0.9333333333333333,
 0.8666666666666667,
 0.9333333333333333,
 0.8666666666666667,
 1.0,
 1.0,
 1.0]